In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/BATH//Dissertaion/Colab
!pip install wandb
!pip install pyDOE
!pip install stable_baselines3

try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

Mounted at /content/drive
/content/drive/My Drive/BATH/Dissertaion/Colab
     |████████████████████████████████| 1.7 MB 4.1 MB/s 
     |████████████████████████████████| 133 kB 48.3 MB/s 
     |████████████████████████████████| 170 kB 50.4 MB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=35a368a71fb62190f479eb36915157b35cfc3ad0a4c1e627009b8f3079c67861
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=08111a9e927e5e7459a72e5447a1166e700ee8a5cf9ac3d18572a23b2a86f6bb
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=

In [ ]:
import torch
from tqdm import tqdm
import numpy as np

class PINN(torch.nn.Module):
    
    def __init__(self, layers = [10,10], device = None, sensor_coords = [[0,0]]):
        super(PINN, self).__init__() # inherit methods from torch
        
        # set to use GPU if available else cpu
        if device is None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else: 
            self.device = device
            
        # set nn architecture:    
        
        # store hidden layer activation function
        self.hidden_activation = torch.nn.Tanh().to(device)
        
        # initialize nn architecture:
        self.input_layer = torch.nn.Linear(3,layers[0]).to(device)
        self.input_activation = torch.nn.Tanh().to(device)
        self.hidden = torch.nn.ModuleList([torch.nn.Linear(layers[i], layers[i+1]).to(device) for i in range(len(layers)-1)])
        self.output_layer = torch.nn.Linear(layers[-1],1).to(device)
        self.output_activation = torch.nn.Tanh().to(device)
        
        self.sensor_coords = np.append(np.flip(sensor_coords), 
                                       np.zeros(len(sensor_coords)).reshape(-1,1), axis = 1)
        
        # init weights and biases:
        
        torch.nn.init.xavier_normal_(self.input_layer.weight.data, gain=1.66)
        torch.nn.init.zeros_(self.input_layer.bias.data)
        
        for i in range(len(layers) - 1):
            
            # set weights from normal distribution 
            torch.nn.init.xavier_normal_(self.hidden[i].weight.data, gain=1.66)
            
            # init biases as zero
            torch.nn.init.zeros_(self.hidden[i].bias.data)
            
        torch.nn.init.xavier_normal_(self.output_layer.weight.data, gain=1.66)
        torch.nn.init.zeros_(self.output_layer.bias.data)
        
        
    def forward(self, x_in): 
        """Feed forward function through neural network."""
        # convert to tensor
        if torch.is_tensor(x_in) != True:         
            x_in = torch.from_numpy(x_in)
        
        # input layer
        x = self.input_layer(x_in)
        x = self.input_activation(x)
        
        # loop through hidden layers
        for i in range(len(self.hidden)):
            x = self.hidden_activation(self.hidden[i](x))
        
        x_out = self.output_layer(x)
        
        return x_out
        
    def MSE(self, y_pred, y_test):
        return torch.mean((y_pred - y_test)**2)
    
    def train_step(self, closure = True):
        '''Takes one train step, called from Train method for number of epochs.'''
        
        if closure:
            self.optimizer.zero_grad()
            
        # thermal diffisivity
        K = self.K
            
        # predict on initial condition w/ nn
        ic_pred = self.forward(self.ic_x)
        self.mse_ic = self.MSE(ic_pred, self.ic_u)
        
        # predict solution to boundary condition
        bc_pred = self.forward(self.x_bc) #[x,t]
        self.mse_u = self.MSE(bc_pred, self.u_bc)
        
        # predict u w/ network
        self.x.requires_grad = True 
        u_pred = self.forward(self.x)
        
        # differentiate using auto grad:
        
        # 1st deriv wrt X = [y,x,t]
        deriv1 = torch.autograd.grad(u_pred,
                                    self.x, #[y,x,t]
                                    torch.ones([self.x.shape[0], 1]).to(self.device),
                                    retain_graph = True,
                                    create_graph = True)[0]
        
        # 2nd deriv wrt X
        deriv2 = torch.autograd.grad(deriv1,
                                    self.x, 
                                    torch.ones(self.x.shape).to(self.device),
                                    create_graph = True)[0]
        
        du_dy, du_dx, du_dt = deriv1[:,[0]], deriv1[:,[1]], deriv1[:,[2]]
        d2u_dy2, d2u_dx2, d2u_dt2  = deriv2[:,[0]], deriv2[:,[1]], deriv2[:,[2]]
        
        # minimize f by incorporating into the loss
        f = du_dt - K * d2u_dx2 - K * d2u_dy2 + self.vel*du_dx + self.vel*du_dy - self.source # == 0
        
        self.mse_f = self.MSE(f, self.f_hat)
        self.loss = self.mse_u + self.mse_f + self.mse_ic
        
        if closure:
            self.loss.backward()
            
        return self.loss
    
    def train(self, x_bc, u_bc, x, ic_x, ic_u,
              learning_rate = 1e-4,
              epochs = int(1e4),
              LBFGS = True,
              K = 0.1,
              max_iter = 1,
              source = 0,
              sensor_values = [[0],[0],[0]],
              flow_velocity = 0):
        
        self.vel = flow_velocity
        
        x = np.append(x, self.sensor_coords, axis = 0) # add coords to end of x coords
        
        self.source = source
        self.K = K
        
        # boundary conditions
        self.x_bc = x_bc if torch.is_tensor(x_bc) else torch.from_numpy(x_bc).float().to(self.device)  
        self.u_bc = u_bc if torch.is_tensor(u_bc) else torch.from_numpy(u_bc).float().to(self.device)
        
        # initial conditions
        self.ic_x = ic_x if torch.is_tensor(ic_x) else torch.from_numpy(ic_x).float().to(self.device)  
        self.ic_u = ic_u if torch.is_tensor(ic_u) else torch.from_numpy(ic_u).float().to(self.device)  
        
        # f_hat(x) = 0
        self.x = x if torch.is_tensor(x) else torch.from_numpy(x).float().to(self.device)
        self.f_hat = torch.zeros(x.shape[0],1).to(self.device) # PDE is minimized to equal zero
        
        self.f_hat[-len(sensor_values):] = torch.tensor(sensor_values) # set last values equal to obs
        
        # store loss history
        self.loss_u = []
        self.loss_f = []
        loss_t = []
        
        if LBFGS == True:
            # same optimizer used in Raissi et al, quasi-Newton method
            self.optimizer = torch.optim.LBFGS(self.parameters(), 
                                  lr=0.1, 
                                  max_iter = max_iter, 
                                  max_eval = None, 
                                  tolerance_grad = 1e-05, 
                                  tolerance_change = 1e-09, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe') 
            self.optimizer.step(self.train_step) # uses closure
        
        # declar adam opt
        optimizer = torch.optim.Adam(self.parameters(), 
                       lr= learning_rate,
                       betas=(0.9, 0.999), 
                       eps=1e-08, 
                       weight_decay=0, 
                       amsgrad=False)
        
        for i in range(epochs): #tqdm
            
            # perform train step
            loss = self.train_step(closure = False)
            loss_t.append(loss)
            
            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        return loss_t
    
    def predict(self, x_test, load_model = None, time = 5):
        
        # uses pretrained model
        if load_model is not None:
            self.load_state_dict(torch.load(load_model))
        
        if torch.is_tensor(x_test) != True: # convert to tensor send to device
            x_test = torch.from_numpy(x_test).float().to(self.device)
        
        # feedforwards input into nn
        u_pred = self.forward(x_test) 
        u_pred = u_pred.cpu().detach().numpy()
        u_pred = u_pred.reshape(100,256,time) 
        
        return u_pred
    
    def error(self, x_test, u_true):
        u_pred = self.forward(x_test)
        return (torch.linalg.norm((u_true-u_pred),2)/torch.linalg.norm(u_true,2)).item() # l2 error

In [ ]:
import cv2

#import libraries
import gym

import time
import wandb
import numpy as np

import torch 
from matplotlib import pyplot as plt
from IPython.display import clear_output
from tqdm import tqdm 
from dataclasses import dataclass
from typing import Any
from models import Model as Stock_NN
from models import ConvModel2 as ConvModel

import matplotlib.pyplot as plt
is_ipython = 'inline' in plt.get_backend()
if is_ipython: from IPython import display
if is_ipython: display.clear_output(wait=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
plt.rcParams['figure.figsize'] = [15, 10]

import numpy as np
import time
from pyDOE import lhs         #Hypercube Sampling
import scipy.io

#Set default dtype to float32
torch.set_default_dtype(torch.float)


In [ ]:
# loads matlab solution from raissi et al, data used in init conditions
y = np.linspace(-1,1,256).reshape(-1,1)
x = np.linspace(0,0.99,100).reshape(-1,1)                                    
usol = np.zeros([256,100])                              

Y, X = np.meshgrid(y,x)                         

ic = usol.copy()

initial_temp = 0

ic.fill(initial_temp)

#plt.pcolormesh(X,Y,ic.T,)
#plt.colorbar()

ic = ic.T.reshape(100,1,256)

timelen = 4

time = np.linspace(0,1, timelen)
time = time.reshape(-1,1)

Y, X, T= np.meshgrid(y,x, time)

In [ ]:
# test data is the 3D coordinates in [y,x,t]
X_u_test = np.hstack((Y.flatten()[:,None], X.flatten()[:,None], T.flatten()[:,None]))

# Domain bounds
lb = X_u_test[0] 
ub = X_u_test[-1] 

In [ ]:



def trainingdata(ic = ic, n_bc = 100, n_coll = 10000, n_ic = 100, temp = 0, 
                 sensor_coords = None, sensor_values = None):

    # initial conditions: 100 x 256 where t = 0
    init_x = np.hstack((Y[:,:,0][:,None], X[:,:,0][:,None], T[:,:,0][:,None]))
    init_u = ic

    #where x = 0 for t and y
    leftedge_x = np.hstack((Y[0,:][:,None], X[0,:][:,None], T[0,:][:,None])) #L1
    leftedge_u = np.array([0]*y.shape[0]).reshape(-1,1) #usol[:,0][:,None] #* initial_temp #np.full([256,1], 0) #np.full([256,1], 0) 

    #where x = 1 for all t and y
    rightedge_x = np.hstack((Y[0,:][:,None], X[-1,:][:,None], T[-1,:][:,None])) #L1
    rightedge_u =  np.array([0]*y.shape[0]).reshape(-1,1) #usol[:,0][:,None] #* initial_temp  #np.full([256,1], 0)# np.full([256,1], 0)#

    #bottom where y = -1
    bottomedge_x = np.hstack((Y[:,0][:,None], X[:,0][:,None], T[:,0][:,None])) #L2
    bottomedge_u = np.array([temp]*x.shape[0]).reshape(-1,1) #usol[-1,:][:,None] #np.full([100,1], 0) #

    #top where y = 1
    topedge_x = np.hstack((Y[:,-1][:,None], X[:,0][:,None], T[:,-1][:,None])) #L3
    topedge_u = np.array([temp]*x.shape[0]).reshape(-1,1) #usol[0,:][:,None] #np.full([100,1], 0)  #

    all_bc_x = np.vstack([
                               leftedge_x, 
                               rightedge_x, 
                               bottomedge_x, 
                               topedge_x]) 

    all_bc_u = np.vstack([
                             leftedge_u, 
                             rightedge_u, 
                             bottomedge_u, 
                             topedge_u])   

    #choose random n_bc points for training
    idx = np.random.choice(all_bc_x.shape[0], n_bc, replace=False) 

    bc_x = all_bc_x[idx, :] 
    bc_u = all_bc_u[idx,:]     

    id_x = np.random.choice(100, n_ic, replace=False) 
    id_y = np.random.choice(256, n_ic, replace=False)
    
    ic_x = init_x[id_x,:,id_y]
    ic_u = init_u[id_x,:,id_y]

    # create collocation points
    store = []
    for i in range(time.shape[0]):
        coll_points = lb[:2] + (ub[:2] - lb[:2]) * lhs(2,n_coll)
        # assert collocation points have been sampled from the correct range...
        assert((coll_points[:, 1] >= 0.0).all() and (coll_points[:, 1] <= 1.0).all())
        assert((coll_points[:, 0] >= -1.0).all() and (coll_points[:, 1] <= 1.0).all())
        store.append(coll_points) # coll points for every frame t

    # convert to array    
    s = np.array(store)

    t_ = np.array(([time]*n_coll))
    f_x = np.concatenate((s, t_.reshape(time.shape[0], n_coll, 1)), axis = 2).reshape(n_coll, 3, time.shape[0]) 
    
    # flip coordinates # this is done very badly, fix it
    X_u_copy = bc_x.copy()
    ytemp = X_u_copy[:,0,:]
    xtemp = X_u_copy[:,1,:]
    X_u_copy[:,0,:] = xtemp
    X_u_copy[:,1,:] = ytemp
    
    
    f_x = np.vstack((f_x, X_u_copy)) # append boundary coords to collocation coords
    
    if sensor_coords is not None:
        sensor_coords = np.append(np.flip(sensor_coords), np.array([[0],[0],[0]]),axis = 1 )
        ic_x = np.append(ic_x,sensor_coords, axis = 0)
        ic_u = np.append(ic_u, sensor_values, axis = 0)
        
    return f_x, bc_x, bc_u, ic_x, ic_u

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# create the training data
n_bc = 100 # num boundary condition exemplars to sample
n_coll = 10000 # num coll points in each time frame to constrain f
n_ic = 100 # num init condition exemplars to sample
f_x, bc_x, bc_u, ic_x, ic_u = trainingdata(ic, n_bc, n_coll, n_ic, temp = 0)

In [ ]:
# various neural network architectures
l1 = [50,50,50,50,50,50,50,50,50,50]
l2 = [20,20,20,20,20,20,20, 20]
l3 = [20,50,100,100,100,100,100,100,50,20]


sensor_coords = np.array([[0.1, -0.9],[0.9,0.9],[0.5,0.45]])

  

In [ ]:
# instantiate PINN neural network
pinn = PINN(layers = l2, device = device)

In [ ]:
def add_sensor_vals(u_pred,obs):
    x_idx = np.round(sensor_coords[:,0]*84).reshape(-1,1)
    y_idx = np.round(((sensor_coords[:,1]  - -1) / (1 - -1)) * 84).reshape(-1,1)
    idx = np.append(x_idx,y_idx, axis = 1 ).astype(int) # 0 to 1
    for i in range(u_pred.shape[-1]):
        u_pred[idx[:,0],idx[:,1],i] = obs.reshape(3)
    return u_pred

In [ ]:
test = False

if test:
    pinn.train(bc_x[:,:,1], bc_u, f_x[:,:,1], ic_x, ic_u,
                      epochs = 1, 
                      LBFGS = True,
                      K = .01,
                      source = 20)

    u_pred = pinn.predict(X_u_test, load_model = None, time = timelen)

In [ ]:
import random
class Exp_Replay:
    """Experience replay, old samples are removed beyond specified limit
    stores collection of experience tuples (sars), this combats experience correlation"""
    def __init__(self, buffer_n = int(1e5)):   
        
        self.buffer_n = buffer_n
        self.buffer = [None]*buffer_n
        self.idx = 0
        
    def insert(self, sars):
        i = self.idx % self.buffer_n
        self.buffer[i] = sars
        self.idx +=1 # update index
        
    def sample(self, n_sample):
        if self.idx < self.buffer_n:
            return random.sample(self.buffer[:self.idx], n_sample)
        return random.sample(self.buffer, n_sample)
    
# for data storage
@dataclass
class Sars: # store experience tuples
    state: Any
    action: int
    reward: float
    next_state : Any
    done : bool


In [ ]:
class DQN_Agent:
    
    def __init__(self, env,
                 learning_rate = 1e-4, 
                 discount_rate = 0.99,
                 eps_max = 0.9, 
                 eps_min = 0.01,
                 eps_decay = 1e-6, 
                 boltzman_exploration = False,
                 min_rb_size = int(2e4), 
                 sample_size = 100,
                 model_train_freq = 100,
                 tgt_update_freq = 5000,
                 max_epoch = np.inf, 
                 load_model = None,
                 load_PINN = None,
                 device = 'cuda:0',
                 name = 'Breakout',
                 description = '__'):
        
        self.lr = learning_rate
        self.gamma = discount_rate
        self.eps_max = eps_max
        self.eps_min = eps_min
        self.eps_decay = eps_decay
        
        self.boltzman = boltzman_exploration 
        self.min_rb = min_rb_size
        self.sample_size = sample_size 
        self.model_train_freq = model_train_freq 
        self.tgt_update_freq = tgt_update_freq
        self.max_epoch = max_epoch 
        self.load_model = load_model
        self.load_PINN = load_PINN
        self.device = device
        self.name = name
        self.id = np.nan
        self.descrip = description 

        self.log = {'loss': [],
                    'avg_reward': [],
                    'eps': [],
                    'step_num': [],
                    'PINN':[]}

        self.pinn = PINN(layers = l2, device = device, sensor_coords = sensor_coords) 
        
        # init env
        self.env = env #gym.make('stocks-v0', frame_bound=(15, 200), window_size=15)
        
        return
    
    def choose_action(self, eps):
        
        if self.boltzman: # use boltzman exploration
                logits = self.m(torch.Tensor(self.last_observation).unsqueeze(0).to(self.device))[0]
                action = torch.distributions.Categorical(logits = logits).sample().item()
        else:
            if np.random.random() < eps: # explore action space
                action = self.env.action_space.sample()
            else: # greedy action
                action = self.m(torch.Tensor(self.last_observation)
                           .unsqueeze(0).to(self.device)).max(-1)[-1].item()
        return action
    
    def run_episode(self, episode):
        '''runs one episode in the taining process.'''
        
        # compute decaying exploration rate as a function of episode
        eps = (self.eps_max - self.eps_min) * np.exp(-self.eps_decay*self.step_count) + self.eps_min
        
        
        ic = usol.copy()
        initial_temp = 0
        ic.fill(initial_temp)
        ic = ic.T.reshape(100,1,256)
        
        f_x, bc_x, bc_u, ic_x, ic_u = trainingdata(ic, n_bc, n_coll, n_ic, temp = 0)
        obs = self.env.reset()
        LBFGS = True
        self.pinn.train(bc_x[:,:,0], bc_u, f_x[:,:,0], ic_x, ic_u,
                  epochs = 1, 
                  LBFGS = LBFGS,
                  K = .01)
        u_pred = self.pinn.predict(X_u_test, load_model = None, time = timelen)
        ic = u_pred[:,:,-1].reshape(100,1,256) # save last frame as ic to input next prediction
        u_pred = cv2.resize(u_pred, (84,84)) # resize 
        u_pred = np.transpose(u_pred,(2,0,1))
        
        def f(x):
            return 2/ (1+np.exp(-0.2*x) ) - 1

        
        self.last_observation = u_pred #self.env.reset()
        done = False
        
        prev_action = None
        loss = np.nan
        rolling_reward = 0
        norm_reward = 0
        while not done: # until episode ends
            self.tq.update()
            #print(self.step_num)
            
            # choose action
            action = self.choose_action(eps)
            
            # observe state reward by taking action
            obs, reward, done, info = self.env.step(action)
            rolling_reward += reward # sum reward for episode
            #reward = f(reward)
            norm_reward += reward

            LBFGS = False
            if self.step_count % 150 == 0:
                LBFGS = True
            
            if prev_action == 1:
                a = 20
            elif prev_action == 0:
                a = -20
            else:
                a = 0
            
            f_x, bc_x, bc_u, ic_x, ic_u = trainingdata(ic, n_bc, n_coll, n_ic, temp = a,
                                                      sensor_coords = sensor_coords,
                                                      sensor_values = obs)
            #print(obs[2])
            
            prev_action = action
            self.pinn.train(bc_x[:,:,0], bc_u, f_x[:,:,0], ic_x, ic_u,
                  epochs = 1, 
                  LBFGS = LBFGS,
                  K = .01,
                  max_iter = 1,
                  source = np.mean(obs),
                  sensor_values = obs) # a
            
            u_pred = self.pinn.predict(X_u_test, load_model = None, time = timelen)
            
            ic = u_pred[:,:,-1].reshape(100,1,256)
            u_pred = cv2.resize(u_pred, (84,84))
            u_pred = add_sensor_vals(u_pred,obs)
            u_pred = np.transpose(u_pred,(2,0,1))
            #self.log['PINN'].append(u_pred)
            
            
            obs = u_pred
            
            # insert experience tuple at top of buffer
            self.rb.insert(Sars(self.last_observation, action, reward, obs, done))

            self.last_observation = obs # update observation
            
            #  counters
            self.steps_since_train += 1
            self.step_num += 1
            self.step_count += 1
            
            
            
            
            # train prediction network
            if  self.steps_since_train > self.model_train_freq and self.rb.idx > self.min_rb:
                #print('Logging')
                # train model neural network
                loss = self.train_NN(self.m, 
                                     self.rb.sample(self.sample_size), 
                                     self.tgt,
                                     self.env.action_space.n,
                                     self.device)
                self.steps_since_train = 0 # reset train counter
                
                
                wandb.log({'loss': loss.detach().cpu().item(), 
                           'epsilon': eps, 
                           'avg_reward': self.episode_rewards[-1],
                           'norm_reward': self.norm_reward}, 
                          step = self.step_num)
                self.save_reward = np.mean(self.episode_rewards[-1])

                self.epochs_since_tgt_update +=1

                # update target nn
                if self.epochs_since_tgt_update > self.tgt_update_freq:
                    self.tgt.load_state_dict(self.m.state_dict())
                    self.epochs_since_tgt_update = 0

                self.epoch += 1  
            
            #self.log['loss'].append(loss.detach().cpu().item())
            self.log['avg_reward'].append(rolling_reward)
            self.log['eps'].append(eps)
            self.log['step_num'].append(self.step_num)
        self.norm_reward = norm_reward  
        return rolling_reward # return episode rewards
    
    def train_NN(self, 
                 model,
                 transition, 
                 tgt, 
                 num_actions, 
                 device):
        '''trains model passed'''
        
        curr_states = torch.stack([torch.Tensor(s.state) for s in transition]).to(device)
        rewards = torch.stack([torch.Tensor([s.reward]) for s in transition]).to(device)
        next_states = torch.stack([torch.Tensor(s.next_state) for s in transition]).to(device)
        actions = [s.action for s in transition]
        if_done = torch.stack([torch.Tensor([0]) if s.done else torch.Tensor([1]) for s in transition]).to(device)
        
        with torch.no_grad(): # get best next actions with target network
            next_qvals = tgt(next_states).max(-1)[0] #(N, num_actions)

        model.opt.zero_grad()
        qvals = model(curr_states) # shape: (N, num_actins), get qvals of current state
        H_actions = torch.nn.functional.one_hot(torch.LongTensor(actions), num_actions).to(device)
        
        # MSE loss function
        #loss = ((rewards + if_done[:,0]*next_qvals - torch.sum(qvals * H_actions, -1))**2).mean()

        f_loss = torch.nn.SmoothL1Loss()
        target = torch.sum(qvals * H_actions, -1)
        inputs = rewards.squeeze() + if_done[:,0]*self.gamma*next_qvals # Bellman optimality
        loss = f_loss(target, inputs )
        loss.backward()
        model.opt.step()

        return loss
    
    def train(self, identifier = 0, episode_num = 0):
        '''begin training by running episodes until max or interrupted'''
        
        self.id = identifier
        
        if self.name is None:
          self.name = str(self.descrip) + '_eps_' + str(self.eps_max) + '_rb_' + str(self.min_rb) + '_samples_' + str(self.sample_size) + '_tgt_' + str(self.tgt_update_freq) + '_id_' + str(self.id) + '_.pth'

        # init w and b for data viz in dashboard
        wandb.init(project = "Dissertation_Final", name = self.name)    
        
        #instantiate PINN
        
        self.pinn.train(bc_x[:,:,0], bc_u, f_x[:,:,0], ic_x, ic_u,
                  epochs = 1, 
                  LBFGS = False,
                  K = .01)
        u_pred = self.pinn.predict(X_u_test, load_model = None, time = timelen)
        u_pred = cv2.resize(u_pred, (84,84))
        u_pred = u_pred.reshape(4,84,84)
        
        if self.load_PINN is not None:
            self.pinn.load_state_dict(torch.load(self.load_PINN))
        
        # instantiate prediction network
        self.m = ConvModel(u_pred.shape, #self.env.observation_space.shape,
                           self.env.action_space.n).to(self.device)
        if self.load_model is not None:
            self.m.load_state_dict(torch.load(self.load_model))
        
        # instantiate target network
        self.tgt = ConvModel(u_pred.shape, #self.env.observation_space.shape, 
                        self.env.action_space.n).to(self.device)
        self.tgt.load_state_dict(self.m.state_dict()) 
        
        # instantiate buffer
        self.rb = Exp_Replay()
        
        # init counterstw
        self.epoch = 0
        self.steps_since_train = 0
        self.epochs_since_tgt_update = 0
        self.step_num = -self.min_rb
        self.step_count = 0
        self.episode_rewards = [np.nan]
        episode = episode_num
        
        self.tq = tqdm()
        
        ra = str(np.random.random())[2:8]
        
        try:
            while episode < self.max_epoch:
                #if episode % 5 == 0: print (episode)
                self.episode_rewards.append(self.run_episode(episode))
                episode += 1
                
                if episode % 50 == 0:
                    self.cnn_model = f"models/CNN_3D_Diffusion_{episode}_episodes_{self.id}.pth"
                    self.pinn_model = f"models/PINN_3D_Diffusion_{episode}_episodes_{self.id}.pth"
                    torch.save(self.tgt.state_dict(), self.cnn_model)
                    torch.save(self.pinn.state_dict(), self.pinn_model)
                
                
            r = str(np.random.random())[2:8] # random marker
            self.cnn_model = f"models/CNN_3D_Diffusion_{self.step_count}_{r}_{self.id}.pth"
            self.pinn_model = f"models/PINN_3D_Diffusion_{self.step_count}_{r}_{self.id}.pth"
            torch.save(self.tgt.state_dict(), self.cnn_model)
            torch.save(self.pinn.state_dict(), self.pinn_model)
            print('Training Completed')
        except KeyboardInterrupt: # save model on interrupt
            r = str(np.random.random())[2:8]
            self.cnn_model = f"models/CNN_3D_Diffusion_{self.step_count}_{r}_{self.id}.pth"
            self.pinn_model = f"models/PINN_3D_Diffusion_{self.step_count}_{r}_{self.id}.pth"
            torch.save(self.tgt.state_dict(), self.cnn_model)
            torch.save(self.pinn.state_dict(), self.pinn_model)
            print('Training Interrupted')
        
    
    def test_episode(self, env = None, plot = False, load_model = None, load_PINN = None):
        
        if env is not None:
            self.env = env
            #self.env.step(1)
            #self.env.reset()
            
        done = False
        
        tq = tqdm()
        
        
        
        if load_PINN is not None:
            self.pinn = PINN(layers = l2, device = device, sensor_coords = sensor_coords)
            self.pinn.load_state_dict(torch.load(load_PINN))
        
        ic = usol.copy()
        initial_temp = 0
        ic.fill(initial_temp)
        ic = ic.T.reshape(100,1,256)
        
        f_x, bc_x, bc_u, ic_x, ic_u = trainingdata(ic, n_bc, n_coll, n_ic, temp = 0)
        pinn.train(bc_x[:,:,0], bc_u, f_x[:,:,0], ic_x, ic_u,
                  epochs = 1, 
                  LBFGS = False,
                  K = .01)
    
    
        u_pred = pinn.predict(X_u_test, load_model = None, time = timelen)
        ic = u_pred[:,:,-1].reshape(100,1,256)
        u_pred = cv2.resize(u_pred, (84,84))
        u_pred = np.transpose(u_pred,(2,0,1))
        
        
        
        
        
        
        if load_model is not None:
            # instantiate prediction network
            self.m = ConvModel(u_pred.shape, #self.env.observation_space.shape,
                           self.env.action_space.n).to(self.device)
            self.m.load_state_dict(torch.load(load_model))
            
        frames = []
        obs = u_pred 
        self.env.reset()
        
        idx = 0
        reward = 0
        done = False
        
        prev_action = None
        
        counter = 0
        while not done:
            
            #print(self.env.counter)
            
            action = self.m(torch.Tensor(obs).unsqueeze(0).to(self.device)).max(-1)[-1].item()
            obs, r, done, _ = self.env.step(action)
                
            if prev_action == 1:
                a = 20
            elif prev_action == 0:
                a = -20
            else:
                a = 0
            
            f_x, bc_x, bc_u, ic_x, ic_u = trainingdata(ic, n_bc, n_coll, n_ic, temp = a,
                                                      sensor_coords = sensor_coords,
                                                      sensor_values = obs)
            prev_action = action
            
            
            pinn.train(bc_x[:,:,0], bc_u, f_x[:,:,0], ic_x, ic_u,
                  epochs = 1, 
                  LBFGS = False,
                  K = .01,
                 source = np.mean(obs),
                 sensor_values = obs)
            u_pred = pinn.predict(X_u_test, load_model = None, time = timelen)
            


            ic = u_pred[:,:,-1].reshape(100,1,256)
            u_pred = cv2.resize(u_pred, (84,84))
            u_pred = add_sensor_vals(u_pred,obs)
            u_pred = np.transpose(u_pred,(2,0,1))
            
            self.log['PINN'].append(u_pred)
            obs = u_pred
            reward += r
            
            if plot:
                a = self.env.render()
                #plt.figure()
                #plt.imshow(a)
                #plt.show()
                #time.sleep(1)
                clear_output(wait=True)
            else:
                tq.update()
                clear_output(wait=True)
                
            clear_output(wait=True)
            counter += 1
        return reward
    

In [ ]:
ic = usol.copy()
initial_temp = 0
ic.fill(initial_temp)
ic = ic.T.reshape(100,1,256)

from environment import heat_diffusion

env = heat_diffusion(dt = 1e-1, sensor_coords = sensor_coords, 
                     noisy_IC = True, 
                     continuous = False,
                     noisy_source = False,
                     norm_reward = True,
                     scale_reward = False,
                     Cp = 0.09)

0it [00:00, ?it/s]

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [ ]:
print('hello')

hello


In [ ]:
Agent = DQN_Agent(env,
                 learning_rate = 1e-4, 
                 discount_rate = 0.99,
                 eps_max = 0.5, 
                 eps_min = 0.05,
                 eps_decay = 6e-5, 
                 boltzman_exploration = False,
                 min_rb_size = 10000, #200, 
                 sample_size = 500, #300
                 model_train_freq = 5,
                 tgt_update_freq = 1000,
                 max_epoch = 2000, #100, 
                 load_PINN = None,
                 load_model = None,
                 device = 'cuda:0',
                 name = None,
                 description = 'Diffusion_PINN_new_env_'+str(78696))
  
r = str(np.random.random())[2:8] # random marker
print(r)
Agent.train(identifier = r, episode_num = 0)


699144


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc



0it [00:00, ?it/s]
1it [00:00,  2.95it/s]

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


Streaming output truncated to the last 5000 lines.
35314it [2:40:41,  3.89it/s]
35315it [2:40:42,  3.84it/s]
35316it [2:40:42,  3.51it/s]
35317it [2:40:42,  3.64it/s]
35318it [2:40:42,  3.73it/s]
35319it [2:40:43,  3.79it/s]
35320it [2:40:43,  3.81it/s]
35321it [2:40:43,  3.84it/s]
35322it [2:40:44,  3.45it/s]
35323it [2:40:44,  3.58it/s]
35324it [2:40:44,  3.69it/s]
35325it [2:40:44,  3.72it/s]
35326it [2:40:45,  3.77it/s]
35327it [2:40:45,  3.82it/s]
35328it [2:40:45,  3.53it/s]
35329it [2:40:45,  3.65it/s]
35330it [2:40:46,  3.71it/s]
35331it [2:40:46,  3.80it/s]
35332it [2:40:46,  3.82it/s]
35333it [2:40:46,  3.91it/s]
35334it [2:40:47,  3.55it/s]
35335it [2:40:47,  3.53it/s]
35336it [2:40:47,  3.45it/s]
35337it [2:40:48,  3.50it/s]
35338it [2:40:48,  3.61it/s]
35339it [2:40:48,  3.67it/s]
35340it [2:40:49,  3.32it/s]
35341it [2:40:49,  3.47it/s]
35342it [2:40:49,  3.56it/s]
35343it [2:40:49,  3.68it/s]
35344it [2:40:50,  3.69it/s]
35345it [2:40:50,  3.71it/s]
35346it [2:40:50,  3.

In [ ]:
r = str(np.random.random())[2:8] # random marker
Agent.train(identifier = r, episode_num = 0)

In [ ]:
avg_scores = []
i = 54
while True:
    scores = []
    Agent.train(identifier = i)
    i += 1
    print('Testing')
    print(Agent.pinn_model)
    
    
    for _ in range(5):
        env.reset()
        scores.append(Agent.test_episode(plot=False, env = env,
                                    load_PINN = Agent.pinn_model,
                                    load_model = Agent.cnn_model))
        print('scores: ',scores)
    print(Agent.pinn_model) 
    Agent = DQN_Agent(env,
                 learning_rate = 1e-4, 
                 discount_rate = 0.99,
                 eps_max = 0.5, 
                 eps_min = 0.01,
                 eps_decay = 1e-5, 
                 boltzman_exploration = False,
                 min_rb_size = 1000, #100, 
                 sample_size = 100, #10
                 model_train_freq = 50,
                 tgt_update_freq = 400,
                 max_epoch = 50, #100, 
                 load_PINN = Agent.pinn_model,#'models/PINN_3D_Diffusion_10050_325791_67.pth',
                 load_model = Agent.cnn_model,#'models/CNN_3D_Diffusion_10050_325791_67.pth',
                 device = 'cuda:0',
                 name = 'Heat_diffusion_PINN_3D')
    
    
    avg_scores.append(np.mean(scores))
    print(i, ' scores: ', avg_scores)

In [ ]:
plt.plot(np.arange(len(Agent.log['avg_reward'][:])), Agent.log['avg_reward'][:])

In [ ]:
env.reset()

In [ ]:
env.is_done()

In [ ]:
env.counter

In [ ]:
all(np.abs(env.info['energy'][-20:]) < 0.5)

In [ ]:
Agent.test_episode(plot=True, env = env,
                 load_PINN = 'models/PINN_3D_Diffusion_10050_325791_67.pth',
                 load_model = 'models/CNN_3D_Diffusion_10050_325791_67.pth')

In [ ]:
'models/PINN_3D_Diffusion_29473_378029_.pth'

In [ ]:
'models/PINN_3D_Diffusion_2010_004837_47.pth'
'models/CNN_3D_Diffusion_2010_004837_47.pth'

In [ ]:
'models/CNN_3D_Diffusion_8899_186967_s.pth'
'models/PINN_3D_Diffusion_8899_186967_s.pth'

In [ ]:
plt.plot(np.arange(len(env.info['reward'][-200:])), env.info['reward'][-200:])
plt.xlabel('Steps in Episode', size = 30)
plt.ylabel('Reward', size = 30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
#plt.savefig('images/OG_50_epsiodes_3894374.png', dpi = 500)

In [ ]:
plt.plot(np.arange(len(env.info['energy'][-200:])), env.info['energy'][-200:])
plt.xlabel('Steps in Episode', size = 30)
plt.ylabel('Reward', size = 30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
#plt.savefig('images/OG_50_epsiodes_3894374.png', dpi = 500)

In [ ]:
from time import sleep
for i in range(4):
    plt.imshow(Agent.log['PINN'][122][i,:,:],vmin = -5, vmax = 5)
    
    plt.colorbar()
    plt.show()
    sleep(1)
    clear_output(wait=True)

In [ ]:
len(Agent.log['PINN'])

In [ ]:
plt.imshow(Agent.log['PINN'][89])

In [ ]:
f_x, bc_x, bc_u = gen_data(n_bc,n_coll, temp = 20)
#prev_action = action

loss = pinn.train(bc_x, bc_u, f_x,
          epochs = 1, 
          LBFGS = True) 
u_pred = pinn.predict(test)

In [ ]:
plt.imshow(u_pred)
plt.colorbar()

In [ ]:
obs, r, done, _ = env.step(1)

In [ ]:
np.flip(sensor_coords)

In [ ]:
obs